In [ ]:
#library
import os
from datetime import datetime
import time
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns
import math

import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy import stats
from itertools import product

from plotly.subplots import make_subplots

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf




import warnings
warnings.filterwarnings("ignore")
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
        
        
info = pd.read_csv("/kaggle/input/g-research-crypto-forecasting/asset_details.csv")
ctrain = pd.read_csv("/kaggle/input/g-research-crypto-forecasting/train.csv")


In [ ]:
ctrain.head()

In [ ]:
info.Asset_Name.unique()

In [ ]:
ctrain.columns

In [ ]:
#time frame that we have
pd.to_datetime(ctrain['timestamp'], unit='s')

## Candlestick charts

In [ ]:
#Dictionary for assets
rename_dict = {}
asset_details = info 
for a in asset_details['Asset_ID']:
    rename_dict[a] = asset_details[asset_details.Asset_ID == a].Asset_Name.values[0]

display(rename_dict)

In [ ]:
# Convert timestamp
train  = ctrain.copy()
train['timestamp'] = train['timestamp'].astype('datetime64[s]')
#supplemental_train['timestamp'] = supplemental_train['timestamp'].astype('datetime64[s]')
#example_test['timestamp'] = example_test['timestamp'].astype('datetime64[s]')

#train['date'] = read_DatetimeIndex(train['timestamp']).date
#supplemental_train['date'] = pd.DatetimeIndex(supplemental_train['timestamp']).date
#example_test['date'] = pd.DatetimeIndex(example_test['timestamp']).date

# Resample
train_daily = pd.DataFrame()

for asset_id in asset_details.Asset_ID:
    train_single = train[train.Asset_ID == asset_id].copy()

    train_single_new = train_single[['timestamp','Count']].resample('D', on='timestamp').sum()
    train_single_new['Open'] = train_single[['timestamp','Open']].resample('D', on='timestamp').first()['Open']
    train_single_new['High'] = train_single[['timestamp','High']].resample('D', on='timestamp').max()['High']
    train_single_new['Low'] = train_single[['timestamp','Low']].resample('D', on='timestamp').min()['Low']
    train_single_new['Close'] = train_single[['timestamp','Close']].resample('D', on='timestamp').last()['Close']
    train_single_new['Volume'] = train_single[['timestamp','Volume']].resample('D', on='timestamp').sum()['Volume']
    # train_single_new['VWAP']
    #train_single_new['Target'] = train_single[['timestamp','Target']].resample('D', on='timestamp').mean()['Target']
    train_single_new['Asset_ID'] = asset_id

    train_daily = train_daily.append(train_single_new.reset_index(drop=False))
train_daily = train_daily.sort_values(by = ['timestamp', 'Asset_ID']).reset_index(drop=True)

train_daily = train_daily.pivot(index='timestamp', columns='Asset_ID')[['Count', 'Open', 'High', 'Low', 'Close', 'Volume']]
train_daily = train_daily.reset_index(drop=False)

display(train_daily.head(10))

In [ ]:
#visualize Bitcoin for recent data rows - last 200 rows
crypto_df = ctrain

crypto_df.index = pd.to_datetime(crypto_df.timestamp, unit='s')
btc = crypto_df[crypto_df["Asset_ID"]==1] # Asset_ID = 1 for Bitcoin
btc_mini = btc.iloc[-200:] # Select recent data rows


import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=btc_mini.index, open=btc_mini['Open'], high=btc_mini['High'], low=btc_mini['Low'], close=btc_mini['Close'])])
fig.show()

## Training Data Distribution among differnet Assets 

In [ ]:
data = ctrain

asset_count= []
for i in range(14):
    count = (data["Asset_ID"]==i).sum()
    asset_count.append(count)
fig = px.bar(x = asset_details.sort_values("Asset_ID")["Asset_Name"],
             y = asset_count , 
             color = asset_count ,
             color_continuous_scale="Emrld") 
fig.update_xaxes(title="Assets")
fig.update_yaxes(title = "Number of Rows")
fig.update_layout(showlegend = True,
    title = {
        'text': 'Data Distribution ',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

1. Data visualtion for 14 popular cryptocurrency
2. Price History for selected individual cryptocurrency
3. Basic Arima Model for price prediction

## Time history of 3 coins and its return

ref: https://www.kaggle.com/fangya/cryptocurrency-data-visualization-arima

In [ ]:
# Impute Missing Time Value

def c_time_sub(asset_id, data=ctrain):
    df=data[ctrain["Asset_ID"]==asset_id].set_index("timestamp")
    df=df.reindex(range(df.index[0],df.index[-1]+60,60), method="pad")
    df.index = pd.to_datetime(df.index, unit='s')
    return df


#  Bitcoin
btc=c_time_sub(asset_id=1)

#  Ethereum
eth=c_time_sub(asset_id=6)

#  Cardano
ada=c_time_sub(asset_id=3)


In [ ]:
btc.head()

## Cryptocurrency Log Return Correlation Plot for 2021

In [ ]:
import time

# define function to compute log returns
def log_return(series, periods=1):
    return np.log(series).diff(periods=periods)
def plot_corr_2021(ctrain, info):
    # auxiliary function, from datetime to timestamp
    totimestamp = lambda s: np.int32(time.mktime(datetime.strptime(s, "%d/%m/%Y").timetuple()))
    # create dataframe with returns for all assets
    
    crypto_df = ctrain
    all_assets_2021 = pd.DataFrame([])
    
    for asset_id, asset_name in rename_dict.items():
        print(asset_name)
        asset = crypto_df[crypto_df["Asset_ID"]==asset_id].set_index("timestamp")
        asset = asset.loc[totimestamp('01/01/2021'):]
        asset = asset.reindex(range(asset.index[0],asset.index[-1]+60,60),method='pad')
        lret = log_return(asset.Close.fillna(0))[1:]
        
        all_assets_2021 = all_assets_2021.join(lret, rsuffix=asset_name, how="outer")
    all_assets_2021.rename(columns = {'Close':'CloseBitcoin Cash'}, inplace=True)
    sns.heatmap(all_assets_2021.corr());
    """plt.imshow(all_assets_2021.corr());
    plt.yticks(info.Asset_ID.values, info.Asset_Name.values);
    plt.xticks(info.Asset_ID.values, info.Asset_Name.values, rotation='vertical');
    plt.colorbar();
    plt.savefig('corr.jpg')"""
    all_assets_2021_corr =  all_assets_2021.corr()
    """all_assets_2021_corr.columns = info.Asset_Name.values
    all_assets_2021_corr.index = info.Asset_Name.values"""
    return all_assets_2021, all_assets_2021_corr
all_assets_2021, all_assets_2021_corr = plot_corr_2021(ctrain, info)

In [ ]:
all_assets_2021_corr.columns

In [ ]:
#the top correlation
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5, ascending=False):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=ascending)
    return au_corr[0:n]
pd.DataFrame(get_top_abs_correlations(all_assets_2021_corr, n=10, ascending=False), columns=['Correlation'])


In [ ]:
#least correlated
pd.DataFrame(get_top_abs_correlations(all_assets_2021_corr, n=10, ascending=True), columns=['Correlation'])


In [ ]:
rename_dict[2]


In [ ]:
#correlations over time
# join two asset in single DataFrame
def corr_over_time(crypto_df, asset_id_1 =1, asset_id_2 = 6):
    btc = crypto_df[crypto_df["Asset_ID"]==asset_id_1].set_index("timestamp")
    eth= crypto_df[crypto_df["Asset_ID"]==asset_id_2].set_index("timestamp")

    lret_btc_long = log_return(btc.Close)[1:]
    lret_eth_long = log_return(eth.Close)[1:]
    lret_btc_long.rename('lret_btc', inplace=True)
    lret_eth_long.rename('lret_eth', inplace=True)
    two_assets = pd.concat([lret_btc_long, lret_eth_long], axis=1)

    # group consecutive rows and use .corr() for correlation between columns
    corr_time = two_assets.groupby(two_assets.index//(10000*60)).corr().loc[:,"lret_btc"].loc[:,"lret_eth"]
    return corr_time

    #corr_time.plot();
    """plt.xticks([])
    plt.ylabel("Correlation")
    plt.title("Correlation between BTC and ETH over time");"""

fig, ax = plt.subplots(nrows=14, ncols=14, figsize=(30, 25))
for i in range(0,13):
    for j in range(i+1,14):
        g = sns.lineplot(data = corr_over_time(ctrain,asset_id_1 =i, asset_id_2 = j),ax=ax[i,j])
        g.set(xlabel=rename_dict[i], ylabel=rename_dict[j])


The illustration of BTC and ETH prices groupped by year [9] https://www.kaggle.com/iamleonie/to-the-moon-g-research-crypto-forecasting-eda#Data-Overview

The corr changes from time 

### Closing Price for BTC, ETH, ADA

In [ ]:
# Plot the Closing Price for BTC, ETH, ADA (Cardano)
f= plt.figure(figsize=(10,12))  

def gplot(no , data, price, label, ylabel, color):
    ax=f.add_subplot(no)
    plt.plot(data[price], label=label, color=color)
    plt.legend()
    plt.xlabel("Time")
    plt.ylabel(ylabel)
    return plt

gplot(no=311, data=btc, price="Close" , label="BTC 2021 Overall Performance", ylabel="BTC Closing Price", color="Lightskyblue")
gplot(no=312, data=eth, price="Close" ,label="ETH 2021 Overall Performance", ylabel="ETH Closing Price", color="Coral")
gplot(no=313, data=ada, price="Close" ,label="Cardano 2021 Overall Performance", ylabel="ADA Closing Price", color="khaki")

plt.tight_layout()
plt.show()

In [ ]:
#Target : 15 minute resudualized returns
#Residual Return: An asset's residual return equals its excess return minus beta times the benchmark excess return.

f= plt.figure(figsize=(10,12))  
gplot(no=311, data=btc, price="Target" , label="BTC 2021 15min Return Residue", ylabel="BTC residual return", color="Aqua")
gplot(no=312, data=eth, price="Target" ,label="ETH 2021 15min Return Residue", ylabel="ETH residual return", color="Pink")
gplot(no=313, data=ada, price="Target" ,label="ADA 2021 15min Return Residue", ylabel="ADA residual return", color="gold")


plt.tight_layout()
plt.show()

As we can see ETH, and BTC residual return are relatively stable compared to ADA. This might be a good implication that if the investor would take short time trading opportunities, ADA is a better choice.

If the investor is risk averse, BTC or ETH will be a better fit

In [ ]:
#candlestick

def c_chart(data,label):
    candlestick = go.Figure(data = [go.Candlestick(x =data.index, 
                                               open = data[('Open')], 
                                               high = data[('High')], 
                                               low = data[('Low')], 
                                               close = data[('Close')])])
    candlestick.update_xaxes(title_text = 'Time',
                             rangeslider_visible = True)

    candlestick.update_layout(
    title = {
        'text': '{:} Candelstick Chart'.format(label),
        "y":0.8,
        "x":0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

    candlestick.update_yaxes(title_text = 'Price in USD', ticksuffix = '$')
    return candlestick

%matplotlib inline
btc_candle=c_chart(btc[-90:], label="BTC Price")
btc_candle.show()

eth_candle=c_chart(eth[-90:], label="ETH Price ")
eth_candle.show()

## Technical Analysis charts
Across the industry, there are hundreds of patterns and signals that have been developed by researchers to support technical analysis trading, it would be a bit tedious to code them. 
Fortunately there is a technical analysis library in python available(created by <a href="https://github.com/bukosabino" >bukosabino</a>), the library has implemented 42 indicators:


<b>Volume:</b>
<ul>
<li>Money Flow Index (MFI)</li>
<li>Accumulation/Distribution Index (ADI)</li>
<li>On-Balance Volume (OBV)</li>
<li>Chaikin Money Flow (CMF)</li>
<li>Force Index (FI)</li>
<li>Ease of Movement (EoM, EMV)</li>
<li>Volume-price Trend (VPT)</li>
<li>Negative Volume Index (NVI)</li>
<li>Volume Weighted Average Price (VWAP)</li>
</ul>
<b>Volatility:</b>
<ul>
<li>Average True Range (ATR)</li>
<li>Bollinger Bands (BB)</li>
<li>Keltner Channel (KC)</li>
<li>Donchian Channel (DC)</li>
<li>Ulcer Index (UI)</li>
</ul>

<b>Trend:</b>
<ul>
<li>Simple Moving Average (SMA)</li>
<li>Exponential Moving Average (EMA)</li>
<li>Weighted Moving Average (WMA)</li>
<li>Moving Average Convergence Divergence (MACD)</li>
<li>Average Directional Movement Index (ADX)</li>
<li>Vortex Indicator (VI)</li>
<li>Trix (TRIX)</li>
<li>Mass Index (MI)</li>
<li>Commodity Channel Index (CCI)</li>
<li>Detrended Price Oscillator (DPO)</li>
<li>KST Oscillator (KST)</li>
<li>Ichimoku Kinkō Hyō (Ichimoku)</li>
<li>Parabolic Stop And Reverse (Parabolic SAR)</li>
<li>Schaff Trend Cycle (STC)</li>
</ul>

<b>Momentum:</b>
<ul>
<li>Relative Strength Index (RSI)</li>
<li>Stochastic RSI (SRSI)</li>
<li>True strength index (TSI)</li>
<li>Ultimate Oscillator (UO)</li>
<li>Stochastic Oscillator (SR)</li>
<li>Williams %R (WR)</li>
<li>Awesome Oscillator (AO)</li>
<li>Kaufman's Adaptive Moving Average (KAMA)</li>
<li>Rate of Change (ROC)</li>
<li>Percentage Price Oscillator (PPO)</li>
<li>Percentage Volume Oscillator (PVO)</li>
</ul>
<b>Others:</b>
<ul>
<li>Daily Return (DR)</li>
<li>Daily Log Return (DLR)</li>
<li>Cumulative Return (CR)</li>
</ul>



<b>Documentation:</b>
<ul>
    <li><a href = "https://technical-analysis-library-in-python.readthedocs.io/en/latest/">Technical Analysis library documentation</a></li>
</ul>
</div>



## Moving average

In [ ]:
ctrain.head()

In [ ]:
btc.head()

In [ ]:
rename_dict

In [ ]:

def mov_avg(df, a = 'W',Asset_ID =1, N = 10):
    
    df = df[df.Asset_ID == Asset_ID]
    
    df.set_index(df['timestamp'], inplace=True)
    df.drop('timestamp', axis=1, inplace=True)
    df.index = pd.to_datetime(df.index, unit='s')

    #weekly
    convertion={
        'Open':'first',
        'High':'max',
        'Low':'min',
        'Close':'mean',
        'Volume':'sum',    
    }
    ds_df = df.resample(a).apply(convertion)
    #avg # of mins in a week

    
    
    #Simple ma
    ds_df['rolling_mean' + str(a) + '_' + str(5)] = ds_df.Close.rolling(window=5).mean()
    ds_df['rolling_mean' + str(a) + '_' + str(10)] = ds_df.Close.rolling(window=10).mean()
    fig = go.Figure(go.Candlestick(x=ds_df.index,open=ds_df['Open'],high=ds_df['High'],low=ds_df['Low'],close=ds_df['Close']))
    fig.update_layout(title='SMA '+str(rename_dict[Asset_ID])+' Close', yaxis_title=str(rename_dict[Asset_ID]))
    fig.update_yaxes(type="log")
    fig.add_trace(go.Scatter(x=ds_df.index, y=ds_df['Close'],mode='lines',name='Close'))
    fig.add_trace(go.Scatter(x=ds_df.index, y=ds_df['rolling_mean' + str(a) + '_' + str(5)], mode='lines', name='SMA MEAN of 5 ' + str(a),line=dict(color='royalblue', width=2)))
    fig.add_trace(go.Scatter(x=ds_df.index, y=ds_df['rolling_mean' + str(a) + '_' + str(10)], mode='lines', name='SMA MEAN of 10 ' + str(a), line=dict(color='#555555', width=2)))
    fig.show()
    
    
    #Exp ma
    ewma = pd.Series.ewm
    ds_df['rolling_ema_'+ str(N)]  = ds_df.Close.ewm(min_periods=N, span=N).mean()
    fig = go.Figure(go.Candlestick(x=ds_df.index,open=ds_df['Open'],high=ds_df['High'],low=ds_df['Low'],close=ds_df['Close']))
    fig.update_layout(title='EMA ' +str(rename_dict[Asset_ID])+' Close', yaxis_title=str(rename_dict[Asset_ID]))
    fig.update_yaxes(type="log")
    fig.add_trace(go.Scatter(x=ds_df.index, y=ds_df['Close'],mode='lines',name='Close'))
    fig.add_trace(go.Scatter(x=ds_df.index, y=ds_df['rolling_ema_' + str(N)], mode='lines', name='EMA MEAN span of '+str(N)+' rolled by '+ str(a),line=dict(color='royalblue', width=2)))
    fig.show()
    
    
 

    
    
    
    
    
    
mov_avg(ctrain, Asset_ID =1 )


In [ ]:
#ETH
mov_avg(ctrain, Asset_ID =6 )

Other technical analysis could be found here https://www.kaggle.com/shakshyathedetector/crypto-prediction-technical-analysis-features#Moving-average
There are:
- Moving average convergence divergence (MACD) is a trend-following momentum indicator that shows the relationship between two moving averages of prices. The MACD is calculated by subtracting the 26-day exponential moving average (EMA) from the 12-day EMA
- Bollinger Bands are a type of statistical chart characterizing the prices and volatility over time of a financial instrument or commodity, using a formulaic method propounded by John Bollinger in the 1980s. Financial traders employ these charts as a methodical tool to inform trading decisions, control automated trading systems, or as a component of technical analysis. Bollinger Bands display a graphical band (the envelope maximum and minimum of moving averages, similar to Keltner or Donchian channels) and volatility (expressed by the width of the envelope) in one two-dimensional chart.
- A Volume Moving Average is the simplest volume-based technical indicator. Similar to a price moving average, a VMA is an average volume of a security (stock), commodity, index or exchange over a selected period of time. Volume Moving Averages are used in charts and in technical analysis to smooth and describe a volume trend by filtering short term spikes and gaps.

## Statistical analysis

In [ ]:
info = pd.read_csv("/kaggle/input/g-research-crypto-forecasting/asset_details.csv")
ctrain = pd.read_csv("/kaggle/input/g-research-crypto-forecasting/train.csv")

In [ ]:
ctrain.head()

In [ ]:
import time

# define function to compute log returns
def log_return(series, periods=1):
    return np.log(series).diff(periods=periods)
def logret_all_assets(ctrain, info):
    # auxiliary function, from datetime to timestamp
    totimestamp = lambda s: np.int32(time.mktime(datetime.strptime(s, "%d/%m/%Y").timetuple()))
    # create dataframe with returns for all assets
    
    crypto_df = ctrain
    all_assets_2021 = pd.DataFrame([])
    
    for asset_id, asset_name in rename_dict.items():
        #print(asset_name)
        asset = crypto_df[crypto_df["Asset_ID"]==asset_id].set_index("timestamp")
        #asset = asset.loc[totimestamp('01/01/2021'):]
        asset = asset.reindex(range(asset.index[0],asset.index[-1]+60,60),method='pad')
        lret = log_return(asset.Close.fillna(0))[1:]
        
        all_assets_2021 = all_assets_2021.join(lret, rsuffix=asset_name, how="outer")
    all_assets_2021.rename(columns = {'Close':'Close Bitcoin Cash'}, inplace=True)
    all_assets_2021.index  = pd.to_datetime(all_assets_2021.index, unit='s')
    return all_assets_2021
all_assets = logret_all_assets(ctrain, info)

In [ ]:
#logret of all assets
all_assets.head()

In [ ]:
all_assets.to_csv('logret_all_years_assets.csv')

In [ ]:
sns.boxplot(data = all_assets)

In [ ]:
all_assets.head()

In [ ]:
stat = all_assets.describe()

In [ ]:
stat

In [ ]:
stat.loc['mean',:].sort_values()

https://www.investopedia.com/articles/investing/102715/computing-historical-volatility-excel.asp
 r (t) = ln (P (t) / P (t-1)) ->  
 
The total return R at time t is:  
R = r1 + r2 + r3 + 2 + ... +rt-1+ rt,



In [ ]:


stat.loc['std',:].sort_values()

For all time minutes logreturn  

In [ ]:
all_assets.describe()

In [ ]:
all_assets.columns

In [ ]:
# CREATE A FUNCTION THAT CALCULATE REALIZED VOLATILITY for btc
# FROM minute-by-minute LOG RETURNS
def realized_volatility_daily(series_log_return):
    """
    Get the daily realized volatility which is calculated as the square root
    of sum of squares of log returns within a specific window interval 
    """
    n = len(series_log_return)
    series_log_return = series_log_return.fillna(0)
    return np.sqrt(np.sum(series_log_return**2)/(n - 1))

k =24
intervals = [ 7*k]
#, 60 *k, 180 *k, 365*k
vols_df = {}

# ITERATE OVER intervals LIST
for i in intervals:
    # GET DAILY LOG RETURNS USING THAT INTERVAL
    vols = all_assets.CloseBitcoin.rolling(window=i)\
                         .apply(realized_volatility_daily).values

    vols_df[i] = vols

# CONVERT vols_df FROM DICTIONARY TO PANDAS DATAFRAME
vols_df = pd.DataFrame(vols_df, columns=intervals, index=df.index)

In [ ]:
vols_df

In [ ]:
# CHANGING MATPLOTLIB STYLE
plt.style.use(['fivethirtyeight'])

fig, ax = plt.subplots(figsize=(18,7))

for i in intervals:
    if i == 7:
        alpha = 0.5
        lw = 1
    else:
        alpha = 1.0
        lw = 2
    ax.plot(vols_df[i], label=f'{i}-Day Interval Realized Volatility', 
            alpha=alpha, lw=lw)

ax.set_title('Realized Volatility Using Different Interval Windows', fontsize=21)

plt.legend(loc='best', prop={'size': 14})
plt.savefig(os.path.join(directory_to_img, 'diff_intervals.png'), 
            dpi=300, bbox_inches='tight')
plt.show();

In [ ]:
sns.boxplot(data = all_assets_2021)

In [ ]:
stat_2021 = all_assets_2021.describe()

In [ ]:
stat_2021.loc['mean',:].sort_values()

In [ ]:
stat_2021.loc['mean',:].sort_values()

Dodgecoin has 

In [ ]:
sns.pairplot(all_assets_2021)

In [ ]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
%config InlineBackend.figure_format='retina'
warnings.filterwarnings( "ignore", module = "matplotlib\..*" )


def vis_pca(all_assets):
    X_std = StandardScaler().fit_transform(all_assets.fillna(0))
    pca = PCA()
    X_std_pca = pca.fit_transform(X_std)
    plot = plt.scatter(X_std_pca[:,0], X_std_pca[:,1])
    #plt.legend(handles=plot.legend_elements()[0])
    plt.show()

    # Plot the explained variances
    features = range(pca.n_components_)
    plt.bar(features, pca.explained_variance_ratio_, color='black')
    plt.xlabel('PCA features')
    plt.ylabel('variance %')
    plt.xticks(features)
    plt.show()
vis_pca(all_assets)

Fig shows that the first one component explain the majority of the variance in our data. 


In [ ]:


vis_pca(all_assets.resample('M').mean())

In [ ]:
vis_pca(all_assets.resample('W').mean())

In [ ]:




X_std = StandardScaler().fit_transform(all_assets.fillna(0))
pca = PCA()
X_std_pca = pca.fit_transform(X_std)
plot = plt.scatter(X_std_pca[:,0], X_std_pca[:,1])
#plt.legend(handles=plot.legend_elements()[0])
plt.show()

# Plot the explained variances
features = range(pca.n_components_)
plt.bar(features, pca.explained_variance_ratio_, color='black')
plt.xlabel('PCA features')
plt.ylabel('variance %')
plt.xticks(features)



## ARIMA Model 


Ref: https://www.kaggle.com/fangya/cryptocurrency-data-visualization-arima

https://www.kaggle.com/iamleonie/to-the-moon-g-research-crypto-forecasting-eda#Data-Overview

In [ ]:
btc.head()

### BTC Seasonal Decomposition


In [ ]:
#freq https://stackoverflow.com/questions/17001389/pandas-resample-documentation
#explanation https://towardsdatascience.com/time-series-decomposition-and-statsmodels-parameters-69e54d035453
plt.rcParams["figure.figsize"]=(15,7)

def season_df(data, label, a):
    df=data.resample(a).mean()
    seasonal_decompose(df.Close).plot()
    print(label)
    return plt.show()
season_df(data=btc, label="BTC Seasonal Decomposition",a ="M")


In [ ]:
season_df(data=btc, label="BTC Seasonal Decomposition",a ="D")

In [ ]:
season_df(data=btc, label="BTC Seasonal Decomposition",a ="H")

In [ ]:

# CALCULATE PRICE RETURNS AS DAILY PERCENTAGE CHANGE USING pct_change()
btc['returns'] = 100 * btc.Close.pct_change().dropna()

# CALCULATE LOG RETURNS BASED ON ABOVE FORMULA
btc['log_returns'] = np.log(btc.Close/btc.Close.shift(1))
btc.dropna(inplace=True)

# PLOT DISTRIBUTION PLOTS OF RETURNS & LOG RETURNS
# AND VISUALLY COMPARE THEM WITH THE STANDARD NORMAL DISTRIBUTION
df = btc.copy()
with sns.axes_style("darkgrid"):
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(18,12))

    axes[0][0].plot(df.returns, color='blue')
    axes[0][0].set_title('Returns')

    sns.distplot(df.returns, norm_hist=True, fit=stats.norm, color='blue',
                bins=50, ax=axes[0][1])
    axes[0][1].set_title('Returns')

    axes[1][0].plot(df.log_returns, color='green')
    axes[1][0].set_title('Log Returns')

    sns.distplot(df.log_returns, norm_hist=True, fit=stats.norm, color='green',
                bins=50, ax=axes[1][1])
    axes[1][1].set_title('Log Returns')
    plt.tight_layout()

    fig.show();

In [ ]:
from scipy.stats import kurtosis, skew

#returns
print( 'returns: excess kurtosis of normal distribution (should be 0): {}'.format( kurtosis(df.returns) ))
print( 'returns: skewness of normal distribution (should be 0): {}'.format( skew(df.returns) ))

#log_returns
print( 'log_returns: excess kurtosis of normal distribution (should be 0): {}'.format( kurtosis(df.log_returns) ))
print( 'log_returns: skewness of normal distribution (should be 0): {}'.format( skew(df.log_returns) ))

- skewness:
returns - positive, log_returns - negative

- positive kurtosis (leptokurtic) - Both Returns & Log Returns show higher peak with thicker tails than the standard normal distribution.


# Box-Cox Transformation for Arima
Sometimes after applying Box-Cox with a particular value of lambda the process may look stationary. It is sometimes possible that even if after applying the Box-Cox transformation the series does not appear to be stationary, diagnostics from ARIMA modeling can then be used to decide if differencing or seasonal differencing might be useful to to remove polynomial trends or seasonal trends respectively. After that the result might be an ARMA model that is stationary. If diagnostics confirm the orders p an q for the ARMA model, the AR and MA parameters can then be estimated.

https://www.kaggle.com/taniaj/cryptocurrency-predictions-with-arima


In [ ]:
btc.head()

In [ ]:
#Box-Cox Transformation
btc_month=btc.resample("M").mean()
btc_month["close_box"], lmbda=stats.boxcox(btc_month.Close)
print("Dickey–Fuller test: p=%f" % adfuller(btc_month.close_box)[1])


The p-value indicates that series is still not stationary.



# Seasonal differentiation 

One method of differencing data is seasonal differencing, which involves computing the difference between an observation and the corresponding observation in the previous year.

In [ ]:
# Seasonal differentiation (12 months)
btc_month['box_diff_seasonal_12'] = btc_month.close_box - btc_month.close_box.shift(12)
print("Dickey–Fuller test: p=%f" % adfuller(btc_month.box_diff_seasonal_12[12:])[1])

The p-value indicates that series is still not stationary.



In [ ]:
# Seasonal differentiation (3 months)
btc_month['box_diff_seasonal_3'] = btc_month.close_box - btc_month.close_box.shift(3)
print("Dickey–Fuller test: p=%f" % adfuller(btc_month.box_diff_seasonal_3[3:])[1])

The p-value indicates that series is stationary as the computed p-value is lower than the significance level alpha = 0.05.

### Regular differentiation 
Sometimes it may be necessary to difference the data a second time to obtain a stationary time series, which is referred to as second order differencing

In [ ]:
btc_month.box_diff_seasonal_12.head()

In [ ]:
# Regular differentiation (second diff for 12 months)
btc_month['box_diff2'] = btc_month.box_diff_seasonal_12 - btc_month.box_diff_seasonal_12.shift(1)

# STL-decomposition
seasonal_decompose(btc_month.box_diff2[13:]).plot()   
print("Dickey–Fuller test: p=%f" % adfuller(btc_month.box_diff2[13:])[1])

plt.show()

The p-value indicates that series is stationary as the computed p-value is lower than the significance level alpha = 0.05.

### Autocorrelation 
Autocorrelation is the correlation of a time series with the same time series lagged. It summarizes the strength of a relationship with an observation in a time series with observations at prior time steps.

We create autocorrelation factor (ACF) and partial autocorrelation factor (PACF) plots to identify patterns in the above data which is stationary on both mean and variance. The idea is to identify presence of AR and MA components in the residuals.

In [ ]:
#autocorrelation_plot(btc_month.close)
plot_acf(btc_month.Close[13:].values.squeeze(), lags=12)

plt.tight_layout()
plt.show()

There is a positive correlation with the first 9 lags that is perhaps significant for the first 3 lags.

A good starting point for the AR parameter of the model may be 3.

Lets try out autocorrelation on the differences

In [ ]:
# Initial approximation of parameters using Autocorrelation and Partial Autocorrelation Plots
ax = plt.subplot(211)
# Plot the autocorrelation function
#sm.graphics.tsa.plot_acf(btc_month.box_diff2[13:].values.squeeze(), lags=48, ax=ax)
plot_acf(btc_month.box_diff2[13:].values.squeeze(), lags=12, ax=ax)
ax = plt.subplot(212)
#sm.graphics.tsa.plot_pacf(btc_month.box_diff2[13:].values.squeeze(), lags=48, ax=ax)
plot_pacf(btc_month.box_diff2[13:].values.squeeze(), lags=12, ax=ax)
plt.tight_layout()
plt.show()

There are not many spikes in the plots outside the insignificant zone (shaded) so there may not be enough information available in the residuals to be extracted by AR and MA models.

### Arima

AutoRegressive Integrated Moving Average

ARIMA models are denoted with the notation ARIMA(p, d, q). These parameters account for seasonality, trend, and noise in datasets:

p - the number of lag observations to include in the model, or lag order. (AR)
d - the number of times that the raw observations are differenced, or the degree of differencing. (I)
q - the size of the moving average window, also called the order of moving average.(MA)

A linear regression model is constructed including the specified number and type of terms, and the data is prepared by a degree of differencing in order to make it stationary, i.e. to remove trend and seasonal structures that negatively affect the regression model. A value of 0 for a parameter indicates to not use that element of the model.

### Parameter Selection
We will iteratively explore different combinations of parameters. For each combination we fit a new ARIMA model with SARIMAX() and assess its overall quality.

We will use the AIC (Akaike Information Criterion) value, returned with ARIMA models fitted using statsmodels. The AIC measures how well a model fits the data while taking into account the overall complexity of the model. A model that fits the data very well while using lots of features will be assigned a larger AIC score than a model that uses fewer features to achieve the same goodness-of-fit. Therefore, we are interested in finding the model that yields the lowest AIC value.

In [ ]:
#start=datetime(2018, 1, 31), end=datetime(2021, 5, 31)

btc_month.index

In [ ]:
qs = range(0, 3)
ps = range(0, 3)
d=1
parameters = product(ps, qs)
parameters_list = list(parameters)
len(parameters_list)

# Model Selection
results = []
best_aic = float("inf")
warnings.filterwarnings('ignore')
for param in parameters_list:
    try:
        model = SARIMAX(btc_month['2018-01-31':'2021-05-31'].close_box, order=(param[0], d, param[1])).fit(disp=-1)
    except ValueError:
        print('bad parameter combination:', param)
        continue
    aic = model.aic
    if aic < best_aic:
        best_model = model
        best_aic = aic
        best_param = param
    results.append([param, model.aic])

In [ ]:
result_table = pd.DataFrame(results)
result_table.columns = ['parameters', 'aic']
print(result_table.sort_values(by = 'aic', ascending=True).head())
print(best_model.summary())

In [ ]:
print("Dickey–Fuller test:: p=%f" % adfuller(best_model.resid[13:])[1])


### Analysis of Results 
The coef column shows the weight (i.e. importance) of each feature and how each one impacts the time series. The P>|z| column informs us of the significance of each feature weight. Here, each weight has a p-value lower or close to 0.05, so it is reasonable to retain all of them in our model.

When fitting seasonal ARIMA models (and any other models for that matter), it is important to run model diagnostics to ensure that none of the assumptions made by the model have been violated. The plot_diagnostics object allows us to quickly generate model diagnostics and investigate for any unusual behavior.

Our primary concern is to ensure that the residuals of our model are uncorrelated and normally distributed with zero-mean. If the seasonal ARIMA model does not satisfy these properties, it is a good indication that it can be further improved.
In the histogram (top right), the KDE line should follow the N(0,1) line (normal distribution with mean 0, standard deviation 1) closely. This is an indication whether the residuals are normally distributed or not.
In the Q-Q-plot the ordered distribution of residuals (blue dots) should follow the linear trend of the samples taken from a standard normal distribution with N(0, 1). Again, this is an indication whether the residuals are normally distributed.

The standardized residual(top left) plot doesn't display any obvious seasonality. 
This is confirmed by the Correlogram (bottom right), which shows that the time series residuals have low correlation with lagged versions of itself. (?)

In [ ]:
best_model.plot_diagnostics(figsize=(15, 12))
plt.show()

In [ ]:
btc_month.head()

Conclusion: We may consider trying to standardise the distribution further. But lets go ahead and do a prediction anyway...

In [ ]:
#one-step ahead predictions
def invboxcox(y,lmbda):
    if lmbda == 0:
        return(np.exp(y))
    else:
        return(np.exp(np.log(lmbda*y+1)/lmbda))
# Prediction
btc_month_pred = btc_month[['Close']]

btc_month_pred['forecast'] = invboxcox(best_model.predict(start=datetime(2018, 1, 31), end=datetime(2021, 9, 21)), lmbda)
plt.figure(figsize=(18,10))
btc_month_pred.Close.plot()
btc_month_pred.forecast.plot(color='r', ls='--', label='Predicted Close')
plt.legend()
plt.title('Bitcoin monthly forecast')
plt.ylabel('USD')
plt.show()

## SARIMAX

In [ ]:
##Addd SARIMAX (week 6)
##Add rmse
##Add other metric that is used in the competition

In [ ]:
# Initial approximation of parameters
from itertools import product
import warnings
warnings.filterwarnings('ignore')
from statsmodels.tsa.statespace.sarimax import SARIMAX


Qs = range(0, 2)
qs = range(0, 3)
Ps = range(0, 3)
ps = range(0, 3)
D=1
d=1
parameters = product(ps, qs, Ps, Qs)
parameters_list = list(parameters)
len(parameters_list)

# Model Selection
results = []
best_aic = float("inf")
warnings.filterwarnings('ignore')
for param in parameters_list:
    try:
#        model = SARIMAX(btc_month.close_box, order=(param[0], d, param[1]), seasonal_order=(param[2], D, param[3], 12)).fit(disp=-1)
        model = SARIMAX(btc_month.close_box, order=(param[0], d, param[1]), seasonal_order=(param[2], D, param[3], 4)).fit(disp=-1)
    except ValueError:
        print('bad parameter combination:', param)
        continue
    aic = model.aic
    if aic < best_aic:
        best_model = model
        best_aic = aic
        best_param = param
    results.append([param, model.aic])

In [ ]:
# Best Models
result_table = pd.DataFrame(results)
result_table.columns = ['parameters', 'aic']
print(result_table.sort_values(by = 'aic', ascending=True).head())
print(best_model.summary())

In [ ]:
print("Dickey–Fuller test:: p=%f" % adfuller(best_model.resid[13:])[1])


In [ ]:
best_model.plot_diagnostics(figsize=(15, 12))
plt.show()

In [ ]:
# STL-decomposition
plt.figure(figsize=(15,7))
plt.subplot(211)
best_model.resid[13:].plot()
plt.ylabel(u'Residuals')
ax = plt.subplot(212)
#sm.graphics.tsa.plot_acf(best_model.resid[13:].values.squeeze(), lags=48, ax=ax)
plot_acf(best_model.resid[13:].values.squeeze(), lags=12, ax=ax)

print("Dickey–Fuller test:: p=%f" % adfuller(best_model.resid[13:])[1])

plt.tight_layout()
plt.show()

In [ ]:
btc_month.columns

In [ ]:
btc_month.index

In [ ]:
# Prediction
btc_month2 = btc_month[['Close']]
date_list = [datetime(2018, 1, 31), datetime(2018, 4, 30), datetime(2018, 5, 31), datetime(2018, 6, 30)]
future = pd.DataFrame(index=date_list, columns= btc_month.columns)
btc_month2 = pd.concat([btc_month2, future])

btc_month2['forecast'] = invboxcox(best_model.predict(start=0, end=75), lmbda)

plt.figure(figsize=(15,7))
btc_month2.Close.plot()
btc_month2.forecast.plot(color='r', ls='--', label='forecast')
plt.legend()
plt.title('Bitcoin Monthly Close Forecast')
plt.ylabel('USD')
plt.savefig('bitcoin_monthly_forecast.png')
plt.show()

In [ ]:
y_forecasted = btc_month2.forecast
y_truth = btc_month2['2015-01-01':'2017-01-01'].Close

# Compute the root mean square error
rmse = np.sqrt(((y_forecasted - y_truth) ** 2).mean())
print('Mean Squared Error: {}'.format(round(rmse, 2)))

### LSTM


In [ ]:
import os
import gc
import traceback
import numpy as np
import pandas as pd
import seaborn as sns
import gresearch_crypto
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd, numpy as np
from tensorflow.keras import layers
import tensorflow_probability as tfp
import tensorflow.keras.backend as K
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import RobustScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
pd.set_option('display.max_columns', None)

In [ ]:
train_df = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/train.csv')

In [ ]:
# Impute Missing Time Value
"""def c_time_sub(asset_id, data=ctrain):
    df=data[ctrain["Asset_ID"]==asset_id].set_index("timestamp")
    df=df.reindex(range(df.index[0],df.index[-1]+60,60), method="pad")
    return df"""

train_df.index=train_df['timestamp']
train = train_df.sort_index()
ind = train.index.unique()
def reindex(df):
    df = df.reindex(range(ind[0],ind[-1]+60,60),method='nearest')
    df = df.fillna(method="ffill").fillna(method="bfill")
    return df
train = train.groupby('Asset_ID').apply(reindex).reset_index(0, drop=True).sort_index()
gc.collect()

In [ ]:
train = train.reset_index(drop = True)
train[['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP', 'Target']] = train[['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP', 'Target']].astype(np.float32)
train['Target'] = train['Target'].fillna(0)

VWAP_max = np.max(train[np.isfinite(train.VWAP)].VWAP)
VWAP_min = np.min(train[np.isfinite(train.VWAP)].VWAP)
train['VWAP'] = np.nan_to_num(train.VWAP, posinf=VWAP_max, neginf=VWAP_min)
df = train[['Asset_ID', 'Target']].copy()

times = dict((t,i) for i,t in enumerate(df.index.unique()))

df['id'] = df.index.map(times)
df['id'] = df['id'].astype(str) + '_' + df['Asset_ID'].astype(str)
ids = df.id.copy()
del df


train = get_features(train)

train_features = [i for i in train.columns if i not in ['Target', 'date', 'timestamp', 'Asset_ID', 'groups']]
train.head()

In [ ]:
train['Target'] = train['Target'].fillna(0)


In [ ]:
train['Target'].head()

In [ ]:
DEVICE = "TPU" #or "GPU"

SEED = 42

EPOCHS = 10
DEBUG = True
N_ASSETS = 1
WINDOW_SIZE = 15
BATCH_SIZE = 1024
PCT_VALIDATION = 10 # last 10% of the data are used as validation set

In [ ]:
train = train_df.copy()

targets = train['Target'].to_numpy().reshape(-1, N_ASSETS)
features = train.columns.drop(['Asset_ID', 'Target','timestamp'])
train = train[features]
train = train.values
train = train.reshape(-1, N_ASSETS, train.shape[-1])

## Dataset creation
Samples with a duration of WINDOW_SIZE records (minutes) will be formed from the train array. Each sample has a target vector corresponding to the final index if WINDOW_SIZE record.

In [ ]:
class sample_generator(keras.utils.Sequence):
    def __init__(self, x_set, y_set, batch_size, length):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.length = length
        self.size = len(x_set)
    def __len__(self): return int(np.ceil(len(self.x) / float(self.batch_size)))
    def __getitem__(self, idx):
        batch_x=[]
        batch_y=[]
        for i in range(self.batch_size):
            start_ind = self.batch_size*idx + i
            end_ind = start_ind + self.length 
            if end_ind <= self.size:
                batch_x.append(self.x[start_ind : end_ind])
                batch_y.append(self.y[end_ind -1])
        return np.array(batch_x), np.array(batch_y)

In [ ]:
X_train, X_test = train[:-len(train)//PCT_VALIDATION], train[-len(train)//PCT_VALIDATION:]
y_train, y_test = targets[:-len(train)//PCT_VALIDATION], targets[-len(train)//PCT_VALIDATION:]

In [ ]:
train_generator = sample_generator(X_train, y_train, length = WINDOW_SIZE, batch_size = BATCH_SIZE)
val_generator = sample_generator(X_test, y_test, length = WINDOW_SIZE, batch_size = BATCH_SIZE)
print(f'Sample shape: {train_generator[0][0].shape}')
print(f'Target shape: {train_generator[0][1].shape}')

## Training
Our model will be trained for the number of FOLDS and EPOCHS you chose in the configuration above. Each fold the model with lowest validation loss will be saved and used to predict OOF and test. Adjust the variable VERBOSE. The variable VERBOSE=1 or 2 will display the training and validation loss for each epoch as text.

In [ ]:
#Correlations for predicted and real
def MaxCorrelation(y_true,y_pred): 
    return -tf.math.abs(tfp.stats.correlation(y_pred,y_true, sample_axis=None, event_axis=None))
def Correlation(y_true,y_pred): 
    return tf.math.abs(tfp.stats.correlation(y_pred,y_true, sample_axis=None, event_axis=None))

#Masked losses
def masked_mse(y_true, y_pred):
    mask = tf.math.not_equal(y_true, 0.)
    y_true_masked = tf.boolean_mask(y_true, mask)
    y_pred_masked = tf.boolean_mask(y_pred, mask)
    return tf.keras.losses.mean_squared_error(y_true = y_true_masked, y_pred = y_pred_masked)


def masked_mae(y_true, y_pred):
    mask = tf.math.not_equal(y_true, 0.)
    y_true_masked = tf.boolean_mask(y_true, mask)
    y_pred_masked = tf.boolean_mask(y_pred, mask)
    return tf.keras.losses.mean_absolute_error(y_true = y_true_masked, y_pred = y_pred_masked)

def masked_cosine(y_true, y_pred):
    mask = tf.math.not_equal(y_true, 0.)
    y_true_masked = tf.boolean_mask(y_true, mask)
    y_pred_masked = tf.boolean_mask(y_pred, mask)
    return tf.keras.losses.cosine_similarity(y_true_masked, y_pred_masked)

In [ ]:
#Long Short-Term Memory layer
def get_squence_model(x):
    x = layers.LSTM(units=32, return_sequences=True)(x)
    return x

#Model
def get_model(n_assets = N_ASSETS):
    x_input = keras.Input(shape=(train_generator[0][0].shape[1], n_assets, train_generator[0][0].shape[-1]))
    branch_outputs = []
    for i in range(n_assets):
        a = layers.Lambda(lambda x: x[:,:, i])(x_input) # Slicing the ith asset:
        a = layers.Masking(mask_value = 0., )(a)
        a = get_squence_model(a)
        a = layers.GlobalAvgPool1D()(a)
        branch_outputs.append(a)
    x = layers.Concatenate()(branch_outputs)
    x = layers.Dense(units = 128)(x)
    out = layers.Dense(units = n_assets)(x)
    model = keras.Model(inputs=x_input, outputs=out)
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3), loss = masked_cosine, metrics=[Correlation])
    return model

model = get_model()
model.summary()

In [ ]:
tf.keras.utils.plot_model(get_model(n_assets=1), show_shapes=True)

In [ ]:
print(features)

In [ ]:
tf.random.set_seed(0)
estop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 7, verbose = 0, mode = 'min',restore_best_weights = True)
scheduler = keras.optimizers.schedules.ExponentialDecay(1e-3, (0.5 * len(X_train) / BATCH_SIZE), 1e-3)
lr = keras.callbacks.LearningRateScheduler(scheduler, verbose = 1)

history = model.fit(train_generator, validation_data = (val_generator), epochs = EPOCHS, callbacks = [lr, estop])


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 8))
histories = pd.DataFrame(history.history)
epochs = list(range(1,len(histories)+1))
loss = histories['loss']
val_loss = histories['val_loss']
Correlation = histories['Correlation']
val_Correlation = histories['val_Correlation']
ax[0].plot(epochs, loss, label = 'Train Loss')
ax[0].plot(epochs, val_loss, label = 'Val Loss')
ax[0].set_title('Losses')
ax[0].set_xlabel('Epoch')
ax[0].legend(loc='upper right')
ax[1].plot(epochs, Correlation, label = 'Train Correlation')
ax[1].plot(epochs, val_Correlation, label = 'Val Correlation')
ax[1].set_title('Correlations')
ax[1].set_xlabel('Epoch')
ax[1].legend(loc='upper right')
fig.show()
gc.collect()

## Submit to Kaggle

In [ ]:
env = gresearch_crypto.make_env()
iter_test = env.iter_test()

for (test_df, sample_prediction_df) in iter_test:
    test_df = get_features(test_df)
    test_data = test_df.merge(example, how='outer', on='Asset_ID').sort_values('asset_order')
    test = np.array(test_data[features].fillna(0))
    test = test.reshape(-1, 1, N_ASSETS, test.shape[-1])
    test_sample = np.hstack([test_sample, test])[:,-1 * WINDOW_SIZE:]
    y_pred = model.predict(test_sample).squeeze().reshape(-1, 1).squeeze()
    test_data['Target'] = y_pred
    for _, row in test_df.iterrows():
        try: sample_prediction_df.loc[sample_prediction_df['row_id'] == row['row_id'], 'Target'] = test_data.loc[test_data['row_id'] == row['row_id'], 'Target'].item()
        except: sample_prediction_df.loc[sample_prediction_df['row_id'] == row['row_id'], 'Target'] = 0
    env.predict(sample_prediction_df)

In [ ]:
#library
import os
from datetime import datetime
import time
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns
import math

import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy import stats
from itertools import product


from statsmodels.graphics.tsaplots import plot_acf, plot_pacf




import warnings
warnings.filterwarnings("ignore")
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
        
        
info = pd.read_csv("/kaggle/input/g-research-crypto-forecasting/asset_details.csv")
ctrain = pd.read_csv("/kaggle/input/g-research-crypto-forecasting/train.csv")


In [ ]:
info = pd.read_csv("/kaggle/input/g-research-crypto-forecasting/asset_details.csv")
ctrain = pd.read_csv("/kaggle/input/g-research-crypto-forecasting/train.csv")


In [ ]:
def preprocess_data(df,asset_id =1):    
    df=df[df["Asset_ID"]==asset_id].set_index("timestamp")
    df=df.reindex(range(df.index[0],df.index[-1]+60,60), method="pad")
    df.index = pd.to_datetime(df.index, unit='s')

    # CALCULATE PRICE RETURNS AS DAILY PERCENTAGE CHANGE USING pct_change()
    df['returns'] = 100 * df.Close.pct_change().dropna()

    # CALCULATE LOG RETURNS BASED ON ABOVE FORMULA
    df['log_returns'] = np.log(df.Close/df.Close.shift(1))

    # DROPPING THE 1ST ROW OF DATA 
    # BECAUSE I SHIFTED IT FORWARD TO CALCULATE RETURNS/LOG RETURNS
    df.dropna(inplace=True)
    
    #The Open/High/Low/Close prices are usually very similar and highly correlated to each other. Therefore, instead of keeping all of them in the dataset, I would add 2 more features:
    df['HL_sprd'] = np.log((df.High - df.Low) / df.Close)
    df['CO_sprd'] = (df.Close - df.Open) / df.Open
    df['Volume'] = np.log(df.Volume)
    features = ['HL_sprd', 'CO_sprd', 'Volume','log_returns']
    x_train_processed = df[features]     
    y_train_processed = df['Target']
    print()
    return df,x_train_processed,y_train_processed

In [ ]:
df, X_btc, y_btc  = preprocess_data(ctrain,1)

In [ ]:
X_btc.head()

In [ ]:
y_btc.head()

In statistics and econometrics, an augmented Dickey–Fuller test (ADF) tests the null hypothesis that a unit root is present in a time series sample.

In [ ]:
from statsmodels.tsa.stattools import adfuller
# LOG RETURNS
adfuller_results = adfuller(btc.log_returns.dropna())

print(f'ADF Statistic: {adfuller_results[0]}')
print(f'p-value: {adfuller_results[1]}')
print('Critical Values:')
for key, value in adfuller_results[4].items():
    print(f'{key}: {value:.4f}')

In [ ]:
# RETURNS
adfuller_results = adfuller(df.returns.dropna())

print(f'ADF Statistic: {adfuller_results[0]}')
print(f'p-value: {adfuller_results[1]}')
print('Critical Values:')
for key, value in adfuller_results[4].items():
    print(f'{key}: {value:.4f}')

Split

In [ ]:
#SPLIT
def custom_split(X_btc,y_btc):   
    PCT_VALIDATION = 10 # last 10% of the data are used as validation set


    X_train, X_test = X_btc[:-len(X_btc)//PCT_VALIDATION], X_btc[-len(X_btc)//PCT_VALIDATION:]
    y_train, y_test = y_btc[:-len(y_btc)//PCT_VALIDATION], y_btc[-len(y_btc)//PCT_VALIDATION:]
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = custom_split(X_btc,y_btc)   

Scaler

In [ ]:
##think about it later


from sklearn.preprocessing import MinMaxScaler



def transform_volatility_to_scaler(scaler, tf_series):
    '''
    Transform a series to a fitted scaler
    '''
    idx = tf_series.index
    output = pd.Series(scaler.transform(tf_series.values.reshape(-1,1))[:,0],
                       index=idx)
    return output

# INITIATE SCALER
scaler = MinMaxScaler()

# FIT SCALER TO CURRENT VOLATILITY IN TRAINING SET
scaler = scaler.fit(X_train.values.reshape(-1,1))

In [ ]:
""""# TRANSFORM TRAINING CURRENT & FUTURE VOLATILITIES 
x_train_scaled = transform_volatility_to_scaler(scaler_vol, x_train)
y_train_scaled = transform_volatility_to_scaler(scaler_vol, y_train)

# TRANSFORMING VALIDATION CURRENT & FUTURE VOLATILITIES
x_val_scaled = transform_volatility_to_scaler(scaler_vol, x_val)
y_val_scaled = transform_volatility_to_scaler(scaler_vol, y_val)

# TRANSFORMING TEST CURRENT & FUTURE VOLATILITIES
x_test_scaled = transform_volatility_to_scaler(scaler_vol, x_test)
y_test_scaled = transform_volatility_to_scaler(scaler_vol, y_test)""""

In [ ]:
# DEFINE ROOT MEAN SQUARED PERCENTAGE ERROR FUNCTION
def RMSPE(y_true, y_pred):
    """
    Compute Root Mean Squared Percentage Error between 2 arrays
    """
    output = np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))
    return output



# DEFINE ROOT MEAN SQUARED ERROR FUNCTION
def RMSE(y_true, y_pred):
    """
    Compute Root Mean Squared Error between 2 arrays
    """
    output = np.sqrt(mse(y_true, y_pred))
    return output

# INITIATE A DATAFRAME FOR MODEL PERFORMANCE TRACKING & COMPARISON 
perf_df = pd.DataFrame(columns=['Model', 'Validation RMSPE', 'Validation RMSE'])

# A FUNCTION THAT LOGS MODEL NAME, RMSE AND RMPSE INTO perf_df
# FOR EASY COMPARISON LATER
def log_perf(y_true, y_pred, model_name):
    perf_df.loc[len(perf_df.index)] = [model_name, 
                                       RMSPE(y_true, y_pred), 
                                       RMSE(y_true, y_pred)]
    return perf_df


# PLOTTING MODEL PREDICTIONS VS. TARGET VALUES
def viz_model(y_true, y_pred, model_name):
    sns.set_context("paper", font_scale=1.7)
    plt.rcParams["axes.grid"] = False

    with sns.axes_style("whitegrid"):
        plt.figure(figsize=(18,7))
        plt.plot(x_val_scaled, color='gray',  ls=':',
                label=f"Scaled Current Daily Volatility")
        
        plt.plot(y_true, color='blue', lw=2, 
                label=f"Target Volatility")
        plt.plot(y_pred, color='orange', lw=2.5,
                label=f'Forecasted Volatility')
        
        # plt.plot(lr_val, color='gray', alpha=0.4,
        #         label='Daily Log Returns')

        plt.title(f'{model_name} \non Validation Data')
        plt.legend(loc='best', frameon=True)



### 4.7. Multivariate 2-Layered Bidirectional LSTM with Dropout

My first Multivariate LSTM model would be relatively simple with 2 hidden layers of Bidirectional LSTM (quite similar to the Univariate structure above). However, I will also include Dropout layers in between. Having more features means the model would be more prone to **overfitting**, and Dropout layers can help reduce that.

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import backend as K
from decimal import Decimal
# CONVERT A SERIES TO MATRIX DATASET

seed = 20


def windowed_dataset(x_series, y_series, lookback_window):
    dataX, dataY = [], []
    for i in range((lookback_window-1), len(x_series)):
        start_idx = x_series.index[i-lookback_window+1]
        end_idx = x_series.index[i]
        a = x_series[start_idx:end_idx].values
        dataX.append(a)
        dataY.append(y_series[end_idx])

    return np.array(dataX), np.array(dataY)


tf.keras.backend.clear_session()

# SET SEED FOR REPRODUCIBILITY
np.random.seed(seed)

n_past = 30
batch_size = 64
n_dims = X_btc.shape[1]

mat_X_train, mat_y_train = windowed_dataset(X_train, y_train, n_past)

### LSTM
version on git https://github.com/ayman-mezghani/ml4f-project/blob/master/g-research.ipynb

In [ ]:
import os
import gc
import traceback
import numpy as np
import pandas as pd
import seaborn as sns
import gresearch_crypto
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd, numpy as np
from tensorflow.keras import layers
import tensorflow_probability as tfp
import tensorflow.keras.backend as K
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import RobustScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
pd.set_option('display.max_columns', None)

In [ ]:
        
asset_details = pd.read_csv("/kaggle/input/g-research-crypto-forecasting/asset_details.csv")
train = pd.read_csv("/kaggle/input/g-research-crypto-forecasting/train.csv")
test = pd.read_csv("/kaggle/input/g-research-crypto-forecasting/example_test.csv")


Feature eng

In [ ]:
#Params
DEBUG = True
EPOCHS = 200
N_ASSETS = 14
WINDOW_SIZE = 15
BATCH_SIZE = 1024
PCT_VALIDATION = 10 # last 10% of the data are used as validation set

In [ ]:
import gc 

def time_change(train):
    # Convert timestamp
    train['timestamp'] = pd.to_datetime(train['timestamp'], unit='s')

    train.index = train.timestamp
    train.sort_index(inplace=True)
    train['is_real'] = True
    ind = train.index.unique()

    def reindex(df):
        res = df.reindex(pd.date_range(ind.min(), ind.max(), freq='min'))
        res['is_real'].fillna(False, inplace=True)
        res['timestamp'] = res.index
        res = res.fillna(method="ffill").fillna(method="bfill")
        return res
    train = train.groupby('Asset_ID').apply(reindex).reset_index(0, drop=True).sort_values(by=['timestamp', 'Asset_ID'])
    gc.collect()
    return train

def vmap_change(train):
    #VWAP  - The average price of the asset over the time interval, weighted by volume.
    VWAP_max = np.max(train[np.isfinite(train.VWAP)].VWAP)
    VWAP_min = np.min(train[np.isfinite(train.VWAP)].VWAP)
    train['VWAP'] = np.nan_to_num(train.VWAP, posinf=VWAP_max, neginf=VWAP_min)
    
    if DEBUG:
        train = train[-500010:]
    return train

def target_train_shape_change(train):
    targets = train['Target'].to_numpy().reshape(-1, N_ASSETS)
    feature_cols = train.columns.drop(['Asset_ID', 'Target', 'timestamp', 'is_real'])

    #non-real data features are set to 0 which are then masked by the model
    for col in feature_cols:
        train[col] = train[col] * train.is_real
        
    train_data = train[feature_cols].values
    train_data = train_data.reshape(-1, N_ASSETS, train_data.shape[-1])
    return targets, train_data



In [ ]:
train.columns

In [ ]:
def get_features(df):
    df_feat = df[['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP']].copy()

In [ ]:
targets, train_data = target_train_shape_change(vmap_change(time_change(train)))

In [ ]:
#Samples with a duration of WINDOW_SIZE records (minutes) will be formed from the train array. 
#Each sample has a target vector corresponding to the final index if WINDOW_SIZE record.

class sample_generator(keras.utils.Sequence):
    def __init__(self, x_set, y_set, batch_size, length):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.length = length
        self.size = len(x_set)
    def __len__(self): return int(np.ceil(len(self.x) / float(self.batch_size)))
    def __getitem__(self, idx):
        batch_x = []
        batch_y = []
        for i in range(self.batch_size):
            start_ind = self.batch_size * idx + i
            end_ind = start_ind + self.length 
            if end_ind <= self.size:
                batch_x.append(self.x[start_ind : end_ind])
                batch_y.append(self.y[end_ind -1])
        return np.array(batch_x), np.array(batch_y)

In [ ]:
X_train, X_test = train_data[:-len(train_data)//PCT_VALIDATION], train_data[-len(train_data)//PCT_VALIDATION:]
y_train, y_test = targets[:-len(train_data)//PCT_VALIDATION], targets[-len(train_data)//PCT_VALIDATION:]

In [ ]:
train_generator = sample_generator(X_train, y_train, length=WINDOW_SIZE, batch_size=BATCH_SIZE)
val_generator = sample_generator(X_test, y_test, length=WINDOW_SIZE, batch_size=BATCH_SIZE)
print(f'Sample shape: {train_generator[0][0].shape}')
print(f'Target shape: {train_generator[0][1].shape}')

## Metrics and Loss functions

In [ ]:
#Correlations for predicted and real
def MaxCorrelation(y_true,y_pred): 
    return -tf.math.abs(tfp.stats.correlation(y_pred, y_true, sample_axis=None, event_axis=None))

def Correlation(y_true,y_pred): 
    return tf.math.abs(tfp.stats.correlation(y_pred, y_true, sample_axis=None, event_axis=None))

#Masked losses
def masked_mse(y_true, y_pred):
    mask = tf.math.not_equal(y_true, 0.)
    y_true_masked = tf.boolean_mask(y_true, mask)
    y_pred_masked = tf.boolean_mask(y_pred, mask)
    return tf.keras.losses.mean_squared_error(y_true=y_true_masked, y_pred=y_pred_masked)

def masked_mae(y_true, y_pred):
    mask = tf.math.not_equal(y_true, 0.)
    y_true_masked = tf.boolean_mask(y_true, mask)
    y_pred_masked = tf.boolean_mask(y_pred, mask)
    return tf.keras.losses.mean_absolute_error(y_true=y_true_masked, y_pred=y_pred_masked)

def masked_cosine(y_true, y_pred):
    mask = tf.math.not_equal(y_true, 0.)
    y_true_masked = tf.boolean_mask(y_true, mask)
    y_pred_masked = tf.boolean_mask(y_pred, mask)
    return tf.keras.losses.cosine_similarity(y_true_masked, y_pred_masked)

## Multivariate 1-Layered LSTM with Gloobal Average Pool

In [ ]:
train_generator[0][0].shape[1]

In [ ]:
#Model
def get_model_1_layer(n_assets=N_ASSETS):
    x_input = keras.Input(shape=(train_generator[0][0].shape[1], n_assets, train_generator[0][0].shape[-1]))
    branch_outputs = []
    
    for i in range(n_assets):
        a = layers.Lambda(lambda x: x[:,:, i])(x_input) # Slicing the ith asset:
        a = layers.Masking(mask_value = 0., )(a)
        a = layers.LSTM(units=32, return_sequences=True)(a)
        a = layers.GlobalAvgPool1D()(a)
        branch_outputs.append(a)
        
    x = layers.Concatenate()(branch_outputs)
    x = layers.Dense(units = 128)(x)
    out = layers.Dense(units = n_assets)(x)
    model = keras.Model(inputs=x_input, outputs=out)
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3), loss=masked_mse, metrics=[Correlation])
    return model

model_1_layer = get_model_1_layer()
model_1_layer.summary()

In [ ]:
tf.keras.utils.plot_model(get_model_1_layer(n_assets=1), show_shapes=True)

In [ ]:
tf.random.set_seed(0)

#Stop training when a monitored metric has stopped improving
estop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, verbose=0, mode='min', restore_best_weights=True)

scheduler = keras.optimizers.schedules.ExponentialDecay(1e-3, (0.5 * len(X_train) / BATCH_SIZE), 1e-3)
lr = keras.callbacks.LearningRateScheduler(scheduler, verbose=1)

In [ ]:
history_1_layer = model_1_layer.fit(train_generator, validation_data=(val_generator), epochs=EPOCHS, callbacks=[lr, estop])



In [ ]:
def plot_training_history(history):
    fig, ax = plt.subplots(1, 2, figsize=(16, 8))
    histories = pd.DataFrame(history.history)
    epochs = list(range(1,len(histories)+1))
    loss = histories['loss']
    val_loss = histories['val_loss']
    correlation = histories['Correlation']
    val_correlation = histories['val_Correlation']
    ax[0].plot(epochs, loss, label='Train Loss')
    ax[0].plot(epochs, val_loss, label='Val Loss')
    ax[0].set_title('Losses')
    ax[0].set_xlabel('Epoch')
    ax[0].legend(loc='upper right')
    ax[1].plot(epochs, correlation, label='Train Correlation')
    ax[1].plot(epochs, val_correlation, label='Val Correlation')
    ax[1].set_title('Correlations')
    ax[1].set_xlabel('Epoch')
    ax[1].legend(loc='upper right')
    fig.show()
gc.collect()



In [ ]:
plot_training_history(history_1_layer)

In [ ]:
predictions_1_layer = model_1_layer.predict(val_generator)

In [ ]:
perf_df = pd.DataFrame(columns=['Model', 'asset', 'corr','weights'])# A FUNCTION THAT LOGS MODEL NAME, RMSE AND RMPSE INTO perf_df

print('Asset:    Corr. coef.')
print('---------------------')
for i in range(N_ASSETS):
    # drop first 14 values in the y_test, since they are absent in val_generator labels
    y_true = np.squeeze(y_test[WINDOW_SIZE - 1:, i])
    y_pred = np.squeeze(predictions_1_layer[:, i])
    real_target_ind = np.argwhere(y_true != 0)
    asset_name = asset_details[asset_details.Asset_ID == i]['Asset_Name'].item()
    weights = asset_details[asset_details.Asset_ID == i]['Weight'].item()
    correl = np.corrcoef(y_pred[real_target_ind].flatten(), y_true[real_target_ind].flatten())[0,1]
    perf_df = perf_df.append([pd.Series(['1 layer LSTM',asset_name,correl,weights ],index = perf_df.columns)], ignore_index=True)
    
    print(f"{asset_name}: {correl:.4f}")
    plt.plot(y_true, label='Target')
    plt.plot(y_pred, label='Prediction')
    plt.xlabel('Time')
    plt.ylabel('Target')
    plt.legend()
    plt.show()

In [ ]:
perf_df

In [ ]:
perf_df = perf_df.append([pd.Series(['1 layer LSTM', 'WEIGTED', (perf_df['corr']*perf_df['weights']).sum(), 1 ],index = perf_df.columns)], ignore_index=True)

In [ ]:
perf_df.to_csv('Performance.csv')

In [ ]:
perf_df

In [ ]:
import datetime
timestamp = datetime.datetime.now()
model_1_layer.save(f"model_1_layer{timestamp.strftime('%s')}")

In [ ]:
model_1_layer.save('model_1_layer.h5')

In [ ]:
# load data that is just before the test set
sup = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/supplemental_train.csv').sort_values(by=['timestamp', 'Asset_ID'])

# only the last 15 min are useful
sup = sup[-WINDOW_SIZE * (N_ASSETS):]

# prepare the sup data. Test data will be appended
test_sample = np.array(get_features(sup))
#test_sample = np.array(sup[feature_cols])
test_sample = test_sample.reshape(-1, (N_ASSETS), test_sample.shape[-1])
test_sample = np.expand_dims(test_sample, axis=0)


env = gresearch_crypto.make_env()
iter_test = env.iter_test()

"""
env = gresearch_crypto.make_env()
iter_test = env.iter_test()
"""
for (test_df, sample_prediction_df) in iter_test:
    # get the features
    test_df = get_features(test_df)
    
    # always sort by time, then by Asset_ID
    test_df = test_df.sort_values(by=['timestamp', 'Asset_ID'])
    
    # to map to the corresponding rows    
    asset_id_row_id_map = {a_id: r_id for a_id, r_id in test_df[['Asset_ID', 'row_id']].values}
    
    # matrix of features
    test = np.array(test_df[feature_cols].fillna(0))
    
    # reshaping (similar to the train)
    test = test.reshape(-1, 1, N_ASSETS, test.shape[-1])
    
    # Stack the test data to previous data, keep only last 15 min 
    test_sample = np.hstack([test_sample, test])[:,-WINDOW_SIZE:]
    
    y_pred = model.predict(test_sample).squeeze().reshape(-1, 1).squeeze()
    
    for i, p in enumerate(y_pred):
        sample_prediction_df.loc[sample_prediction_df['row_id'] == asset_id_row_id_map[i], 'Target'] = p
    env.predict(sample_prediction_df)

## Multivariate 2-Layered Bidirectional LSTM Model

In [ ]:
#Model
def get_model(n_assets=N_ASSETS):
    x_input = keras.Input(shape=(train_generator[0][0].shape[1], n_assets, train_generator[0][0].shape[-1]))
    branch_outputs = []

    for i in range(n_assets):
        a = layers.Lambda(lambda x: x[:,:, i])(x_input) # Slicing the ith asset:
        a = layers.Masking(mask_value = 0., )(a)
        a = layers.Bidirectional(layers.LSTM(32, return_sequences=True))(a)
        a = layers.Bidirectional(layers.LSTM(16))(a)
        #a = layers.GlobalAvgPool1D()(a)
        branch_outputs.append(a)
    
    x = layers.Concatenate()(branch_outputs)
    x = layers.Dense(units = 128)(x)
    out = layers.Dense(units = n_assets)(x)
    model = keras.Model(inputs=x_input, outputs=out)
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3), loss=masked_mse, metrics=[Correlation])
    return model

model = get_model()
model.summary()

tf.keras.utils.plot_model(get_model(n_assets=1), show_shapes=True)

#print(features)

tf.random.set_seed(0)
estop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, verbose=0, mode='min', restore_best_weights=True)
scheduler = keras.optimizers.schedules.ExponentialDecay(1e-3, (0.5 * len(X_train) / BATCH_SIZE), 1e-3)
lr = keras.callbacks.LearningRateScheduler(scheduler, verbose=1)


#train and test


history = model.fit(train_generator, validation_data=(val_generator), epochs=EPOCHS, callbacks=[lr, estop])

plot_training_history(history)

predictions = model.predict(val_generator)



In [ ]:
plot_training_history(history)

In [ ]:
# DEFINE ROOT MEAN SQUARED ERROR FUNCTION
def RMSE(y_true, y_pred):
    """
    Compute Root Mean Squared Error between 2 arrays
    """
    output = np.sqrt(mse(y_true, y_pred))
    return output
# INITIATE A DATAFRAME FOR MODEL PERFORMANCE TRACKING & COMPARISON 
perf_df = pd.DataFrame(columns=['Model', 'Validation RMSPE', 'Validation RMSE'])

# A FUNCTION THAT LOGS MODEL NAME, RMSE AND RMPSE INTO perf_df
# FOR EASY COMPARISON LATER
def log_perf(y_true, y_pred, model_name):
    perf_df.loc[len(perf_df.index)] = [model_name, 
                                       RMSPE(y_true, y_pred), 
                                       RMSE(y_true, y_pred)]
    return perf_df


In [ ]:
y_test

In [ ]:
predictions[:,0]

In [ ]:
print('Asset:    Corr. coef.')
print('---------------------')
for i in range(N_ASSETS):
    # drop first 14 values in the y_test, since they are absent in val_generator labels
    y_true = np.squeeze(y_test[WINDOW_SIZE - 1:, i])
    y_pred = np.squeeze(predictions[:, i])
    real_target_ind = np.argwhere(y_true != 0)
    asset_name = asset_details[asset_details.Asset_ID == i]['Asset_Name'].item()
    print(f"{asset_name}: {np.corrcoef(y_pred[real_target_ind].flatten(), y_true[real_target_ind].flatten())[0,1]:.4f}")
    plt.plot(y_true, label='Target')
    plt.plot(y_pred, label='Prediction')
    plt.xlabel('Time')
    plt.ylabel('Target')
    plt.legend()
    plt.show()

In [ ]:
# load data that is just before the test set
sup = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/supplemental_train.csv').sort_values(by=['timestamp', 'Asset_ID'])

# only the last 15 min are useful
sup = sup[-WINDOW_SIZE * (N_ASSETS):]

# prepare the sup data. Test data will be appended
test_sample = np.array(get_features(sup))
#test_sample = np.array(sup[feature_cols])
test_sample = test_sample.reshape(-1, (N_ASSETS), test_sample.shape[-1])
test_sample = np.expand_dims(test_sample, axis=0)


env = gresearch_crypto.make_env()
iter_test = env.iter_test()

"""
env = gresearch_crypto.make_env()
iter_test = env.iter_test()
"""
for (test_df, sample_prediction_df) in iter_test:
    # get the features
    test_df = get_features(test_df)
    
    # always sort by time, then by Asset_ID
    test_df = test_df.sort_values(by=['timestamp', 'Asset_ID'])
    
    # to map to the corresponding rows    
    asset_id_row_id_map = {a_id: r_id for a_id, r_id in test_df[['Asset_ID', 'row_id']].values}
    
    # matrix of features
    test = np.array(test_df[feature_cols].fillna(0))
    
    # reshaping (similar to the train)
    test = test.reshape(-1, 1, N_ASSETS, test.shape[-1])
    
    # Stack the test data to previous data, keep only last 15 min 
    test_sample = np.hstack([test_sample, test])[:,-WINDOW_SIZE:]
    
    y_pred = model.predict(test_sample).squeeze().reshape(-1, 1).squeeze()
    
    for i, p in enumerate(y_pred):
        sample_prediction_df.loc[sample_prediction_df['row_id'] == asset_id_row_id_map[i], 'Target'] = p
    env.predict(sample_prediction_df)

In [ ]:
test_df.head()

In [ ]:
iter_test = env.iter_test()